In [42]:
import torch

# CL

In [43]:
import torch

def create_graph_coloring_hamiltonian(adj_matrix, num_colors):
    """
    Create an Ising Hamiltonian for the graph coloring problem.
    
    Parameters:
    -----------
    adj_matrix : torch.Tensor
        Adjacency matrix of shape (n, n) where n is the number of nodes.
        adj_matrix[i, j] = 1 if nodes i and j are connected, 0 otherwise.
    num_colors : int
        Number of available colors.
        
    Returns:
    --------
    torch.Tensor
        Symmetric tensor representing the Ising Hamiltonian of shape (n*k, n*k).
    """
    num_nodes = adj_matrix.shape[0]
    hamiltonian_size = num_nodes * num_colors
    
    # Initialize the Hamiltonian matrix with zeros
    H = torch.zeros((hamiltonian_size, hamiltonian_size), dtype=torch.float)
    
    # CONSTRAINT 1: Each node must have exactly one color
    for node in range(num_nodes):
        for color1 in range(num_colors):
            # Calculate the index in our flattened representation
            idx1 = node * num_colors + color1
            
            # Encourage having at least one color (negative diagonal term)
            H[idx1, idx1] = -1
            
            # Penalize having multiple colors (positive cross terms)
            for color2 in range(color1+1, num_colors):
                idx2 = node * num_colors + color2
                # Make it energetically unfavorable to have two colors for one node
                H[idx1, idx2] = 2
                H[idx2, idx1] = 2  # Keep it symmetric
    
    # CONSTRAINT 2: Adjacent nodes cannot have the same color
    for node1 in range(num_nodes):
        for node2 in range(node1+1, num_nodes):
            # Check if nodes are adjacent in the graph
            if adj_matrix[node1, node2] > 0:
                for color in range(num_colors):
                    # Find the indices for these node-color combinations
                    idx_node1 = node1 * num_colors + color
                    idx_node2 = node2 * num_colors + color
                    
                    # Penalize adjacent nodes having the same color
                    H[idx_node1, idx_node2] = 2
                    H[idx_node2, idx_node1] = 2  # Keep it symmetric
    
    return H

def create_graph_coloring_hamiltonian_v2(adj_matrix, n_colors, penalty_node=1.0, penalty_edge=1.0):
    n = adj_matrix.shape[0]
    Q = torch.zeros((n * n_colors, n * n_colors), dtype=torch.float32)

    # One-hot color constraint per node
    for i in range(n):
        for c1 in range(n_colors):
            idx1 = i * n_colors + c1
            Q[idx1, idx1] += penalty_node  # X^2 term
            for c2 in range(c1 + 1, n_colors):
                idx2 = i * n_colors + c2
                Q[idx1, idx2] += 2 * penalty_node
                Q[idx2, idx1] += 2 * penalty_node  # ensure symmetry
            Q[idx1, idx1] += -2 * penalty_node  # from -2*X

    # Adjacent nodes with same color
    for i in range(n):
        for j in range(i+1, n):  # upper triangle, symmetric later
            if adj_matrix[i, j] != 0:
                for c in range(n_colors):
                    idx_i = i * n_colors + c
                    idx_j = j * n_colors + c
                    Q[idx_i, idx_j] += penalty_edge
                    Q[idx_j, idx_i] += penalty_edge  # ensure symmetry
    return Q

def decode_coloring(solution_vector, num_nodes, num_colors):
    """
    Convert a solution vector into node color assignments.
    
    Parameters:
    -----------
    solution_vector : torch.Tensor
        Binary vector of shape (n*k,) representing the solution.
    num_nodes : int
        Number of nodes in the graph.
    num_colors : int
        Number of available colors.
        
    Returns:
    --------
    torch.Tensor
        Color assignment for each node.
    """
    # Reshape the flat solution vector into a matrix
    # Each row represents a node, each column represents a color
    node_color_matrix = solution_vector.reshape(num_nodes, num_colors)
    
    # For each node, find which color has a 1 (or the highest value)
    colors = torch.argmax(node_color_matrix, dim=1)
    return colors

def is_valid_coloring(adj_matrix, coloring):
    """
    Check if a coloring assignment is valid (no adjacent nodes have the same color).
    
    Parameters:
    -----------
    adj_matrix : torch.Tensor
        Adjacency matrix of shape (n, n).
    coloring : torch.Tensor
        Color assignments for each node.
        
    Returns:
    --------
    bool
        True if the coloring is valid, False otherwise.
    """
    num_nodes = adj_matrix.shape[0]
    
    # Check all edges
    for node1 in range(num_nodes):
        for node2 in range(node1+1, num_nodes):
            # If nodes are adjacent and have the same color, the coloring is invalid
            if adj_matrix[node1, node2] > 0 and coloring[node1] == coloring[node2]:
                return False
    
    return True


# Create a simple square graph with 4 nodes
num_nodes = 4
num_colors = 2

# Create adjacency matrix for a square/cycle graph
# 0 -- 1
# |    |
# 3 -- 2
adj_matrix = torch.tensor(
    [[0, 1, 0, 1], # 2
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0]]
)

# Create the Hamiltonian matrix
H = create_graph_coloring_hamiltonian(adj_matrix, num_colors)
H2 = create_graph_coloring_hamiltonian_v2(adj_matrix, num_colors)

# In practice, you would use a solver to minimize the energy
# For this example, we'll manually create a valid solution

# Create a valid coloring: Nodes 0,2 = color 0, Node 1 = color 1, Node 3 = color 2
solution = torch.zeros(num_nodes * num_colors)
solution[0] = 1              # Node 0 has color 0
solution[num_colors+1] = 1   # Node 1 has color 1
solution[2*num_colors] = 1   # Node 2 has color 0
solution[3*num_colors+1] = 1 # Node 3 has color 2

# Calculate the energy of this solution
energy = solution @ H @ solution
print(f"Energy of valid coloring: {energy.item()}")

# Decode and verify the solution
coloring = decode_coloring(solution, num_nodes, num_colors)
print(f"Node colors: {coloring}")
print(f"Is valid: {is_valid_coloring(adj_matrix, coloring)}")



print(solution)

H

Energy of valid coloring: -4.0
Node colors: tensor([0, 1, 0, 1])
Is valid: True
tensor([1., 0., 0., 1., 1., 0., 0., 1.])


tensor([[-1.,  2.,  2.,  0.,  0.,  0.,  2.,  0.],
        [ 2., -1.,  0.,  2.,  0.,  0.,  0.,  2.],
        [ 2.,  0., -1.,  2.,  2.,  0.,  0.,  0.],
        [ 0.,  2.,  2., -1.,  0.,  2.,  0.,  0.],
        [ 0.,  0.,  2.,  0., -1.,  2.,  2.,  0.],
        [ 0.,  0.,  0.,  2.,  2., -1.,  0.,  2.],
        [ 2.,  0.,  0.,  0.,  2.,  0., -1.,  2.],
        [ 0.,  2.,  0.,  0.,  0.,  2.,  2., -1.]])

# CL & CG

In [44]:
# import torch

# def create_graph_coloring_ising_hamiltonian(adj_matrix, k):
#     """
#     Create an Ising Hamiltonian for the graph coloring problem.
    
#     Parameters:
#     -----------
#     adj_matrix : torch.Tensor
#         Adjacency matrix of shape (n, n) where n is the number of nodes.
#         adj_matrix[i, j] = 1 if nodes i and j are adjacent, 0 otherwise.
#     k : int
#         Number of colors.
        
#     Returns:
#     --------
#     torch.Tensor
#         Symmetric tensor representing the Ising Hamiltonian of shape (n*k, n*k).
#     """
#     n = adj_matrix.shape[0]
    
#     # Initialize the Hamiltonian matrix
#     H = torch.zeros((n*k, n*k), dtype=torch.float)
    
#     # Constraint 1: Each node must have exactly one color
#     # Add penalty for a node having multiple colors or no color
#     for i in range(n):
#         for c1 in range(k):
#             idx1 = i * k + c1
            
#             # Diagonal terms for one-hot constraint
#             H[idx1, idx1] = -1
            
#             # Cross terms to penalize multiple colors for the same node
#             for c2 in range(c1+1, k):
#                 idx2 = i * k + c2
#                 H[idx1, idx2] = 2
#                 H[idx2, idx1] = 2
    
#     # Constraint 2: Adjacent nodes cannot have the same color
#     # Add penalty for adjacent nodes having the same color
#     for i in range(n):
#         for j in range(i+1, n):
#             if adj_matrix[i, j] > 0:  # If nodes are adjacent
#                 for c in range(k):
#                     idx_i = i * k + c
#                     idx_j = j * k + c
                    
#                     # Penalty for same color
#                     H[idx_i, idx_j] = 2
#                     H[idx_j, idx_i] = 2
    
#     


# def encode_graph_coloring_ising(adj_matrix, n_colors, penalty_node=1.0, penalty_edge=1.0):
#     n = adj_matrix.shape[0]
#     Q = torch.zeros((n * n_colors, n * n_colors), dtype=torch.float32)

#     # One-hot color constraint per node
#     for i in range(n):
#         for c1 in range(n_colors):
#             idx1 = i * n_colors + c1
#             Q[idx1, idx1] += penalty_node  # X^2 term
#             for c2 in range(c1 + 1, n_colors):
#                 idx2 = i * n_colors + c2
#                 Q[idx1, idx2] += 2 * penalty_node
#                 Q[idx2, idx1] += 2 * penalty_node  # ensure symmetry
#             Q[idx1, idx1] += -2 * penalty_node  # from -2*X

#     # Adjacent nodes with same color
#     for i in range(n):
#         for j in range(i+1, n):  # upper triangle, symmetric later
#             if adj_matrix[i, j] != 0:
#                 for c in range(n_colors):
#                     idx_i = i * n_colors + c
#                     idx_j = j * n_colors + c
#                     Q[idx_i, idx_j] += penalty_edge
#                     Q[idx_j, idx_i] += penalty_edge  # ensure symmetry

#     return Q


# def decode_solution(x, n, k):
#     """
#     Decode the solution from the binary vector.
    
#     Parameters:
#     -----------
#     x : torch.Tensor
#         Binary vector of shape (n*k,) representing the solution.
#     n : int
#         Number of nodes.
#     k : int
#         Number of colors.
        
#     Returns:
#     --------
#     torch.Tensor
#         Coloring assignment of shape (n,) where each element is the color index.
#     """
#     x_reshaped = x.reshape(n, k)
#     colors = torch.argmax(x_reshaped, dim=1)
#     return colors

# def check_solution(adj_matrix, colors):
#     """
#     Check if the coloring is valid.
    
#     Parameters:
#     -----------
#     adj_matrix : torch.Tensor
#         Adjacency matrix of shape (n, n).
#     colors : torch.Tensor
#         Color assignments of shape (n,).
        
#     Returns:
#     --------
#     bool
#         True if the coloring is valid, False otherwise.
#     """
#     n = adj_matrix.shape[0]
    
#     # Check that adjacent nodes have different colors
#     for i in range(n):
#         for j in range(i+1, n):
#             if adj_matrix[i, j] > 0 and colors[i] == colors[j]:
#                 return False
    
#     return True

# # Example usage
# # def example():
# # Create a simple graph with 4 nodes
# n = 4
# k = 2  # 2 colors

# # Create an adjacency matrix for a cycle graph
# adj_matrix = torch.tensor(
#     [[0, 1, 0, 1], # 2
#     [1, 0, 1, 0],
#     [0, 1, 0, 1],
#     [1, 0, 1, 0]]
# )

# # Create the Hamiltonian
# J = create_graph_coloring_ising_hamiltonian(adj_matrix, k)
# print(f"Hamiltonian shape: {J.shape}")

# J2 = encode_graph_coloring_ising(adj_matrix, k)
# print(J2)
# print(J)


# DsB

In [45]:
def run_dsb_simulation(J,  steps=1000, a0=1.0, c0=1.0, dt=0.01):
    """
    Run a (dSB) simulation for solving optimization problems.
    
    Args:
        J: Coupling matrix defining the optimization problem
        steps: Number of simulation steps
        a0: System parameter (constant)
        c0: Coupling strength
        dt: Time step size
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
        x_history: History of position values during simulation
    """
    # Get problem size
    N = J.shape[0]
    
    # Initialize positions and momenta
    x = torch.rand(N, dtype=J.dtype, device=J.device) - 0.5  # Position variables
    y = torch.zeros(N, dtype=J.dtype, device=J.device)       # Momentum variables
    
    # # Initialize history arrays
    # x_history = torch.zeros((steps + 1, N), dtype=J.dtype, device=J.device)
    # x_history[0] = x.clone()
    
    # Define a(t) function - linear increase from 0 to a0
    def a_t_func(t):
        return min(a0 * t / (0.2 * steps * dt), a0)
    
    # Main simulation loop
    for step in range(1, steps + 1):
        t = step * dt
        a_t = a_t_func(t)
        
        # First part of symplectic Euler: update momenta
        # ẏ_i = -[a0 - a(t)]x_i + c0 ∑J_ij*x_j
        y -= dt * ((a0 - a_t) * x - c0 * (torch.matmul(J, torch.sign(x))))
        
        # Second part: update positions
        # ẋ_i = a0 * y_i
        x += dt * a0 * y

        print(x)
        
        # Apply inelastic walls: for any |x_i| > 1
        outside_range = torch.abs(x) > 1.0
        if torch.any(outside_range):
            # Replace with sign (±1)
            x[outside_range] = torch.sign(x[outside_range])
            # Set corresponding momenta to 0
            y[outside_range] = 0.0
        
        # Store current positions
        # x_history[step] = x.clone()
    
    # Get binary solution by taking the sign
    binary_solution = x
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j
    ising_energy = -0.5 * torch.sum(torch.matmul(binary_solution, J) * binary_solution)
    
    return binary_solution, ising_energy#, x_history


binary_solution, ising_energy = run_dsb_simulation(H2)

binary_solution


tensor([-0.4378,  0.3015,  0.2217,  0.1966, -0.3076, -0.4621, -0.3598,  0.2553])
tensor([-0.4372,  0.3012,  0.2215,  0.1967, -0.3077, -0.4618, -0.3596,  0.2547])
tensor([-0.4361,  0.3008,  0.2211,  0.1970, -0.3079, -0.4614, -0.3591,  0.2537])
tensor([-0.4348,  0.3003,  0.2207,  0.1973, -0.3082, -0.4608, -0.3586,  0.2524])
tensor([-0.4330,  0.2997,  0.2200,  0.1977, -0.3086, -0.4600, -0.3579,  0.2508])
tensor([-0.4310,  0.2989,  0.2193,  0.1982, -0.3090, -0.4592, -0.3571,  0.2488])
tensor([-0.4286,  0.2980,  0.2185,  0.1988, -0.3095, -0.4582, -0.3562,  0.2466])
tensor([-0.4258,  0.2969,  0.2175,  0.1994, -0.3100, -0.4570, -0.3551,  0.2440])
tensor([-0.4228,  0.2958,  0.2164,  0.2001, -0.3107, -0.4557, -0.3539,  0.2410])
tensor([-0.4193,  0.2945,  0.2152,  0.2009, -0.3114, -0.4542, -0.3525,  0.2378])
tensor([-0.4156,  0.2931,  0.2138,  0.2018, -0.3121, -0.4527, -0.3510,  0.2342])
tensor([-0.4115,  0.2915,  0.2124,  0.2028, -0.3130, -0.4509, -0.3494,  0.2303])
tensor([-0.4070,  0.2898,  0

tensor([-1., -1., -1., -1., -1., -1., -1., -1.])